                                   Application 1 : Rule Engine with AST

In [1]:
!pip install flask flask-restful


   ---------------------------------------- 0.0/52.8 kB ? eta -:--:--
   ---------------------------------------- 52.8/52.8 kB 2.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
class Node:
    def __init__(self, node_type, value=None, left=None, right=None):
        self.type = node_type  # "operator" or "operand"
        self.value = value      # Optional value for operand nodes
        self.left = left        # Left child
        self.right = right      # Right child

    def __repr__(self):
        return f"Node(type={self.type}, value={self.value}, left={self.left}, right={self.right})"


In [10]:
import re


def split_rule(rule, operator):
    """
    Split the rule into parts based on the specified operator,
    handling nested parentheses correctly.
    """
    parts = []
    depth = 0
    current_part = []

    for token in rule.split():
        if token == '(':
            depth += 1
        elif token == ')':
            depth -= 1
        
        # Check if we are at the top level and found the operator
        if token == operator and depth == 0:
            parts.append(' '.join(current_part).strip())  # Append current part
            current_part = []  # Reset for the next part
        else:
            current_part.append(token)

    # Append the last part if there is any
    if current_part:
        parts.append(' '.join(current_part).strip())

    return parts






    

def create_rule(rule):
    """
    Create a rule node from the given rule string by parsing it
    into an Abstract Syntax Tree (AST).
    """
    print(f"Creating rule: {rule}")
    
    # Base case: if rule is empty or just parentheses
    if not rule or rule.strip() == "":
        return None

    # Remove outer parentheses if present
    rule = rule.strip()
    if rule.startswith('(') and rule.endswith(')'):
        rule = rule[1:-1].strip()

    # Check for AND/OR operators
    if ' AND ' in rule:
        parts = split_rule(rule, 'AND')
        left = create_rule(parts[0].strip())
        right = create_rule(parts[1].strip())
        return Node('operator', 'AND', left, right)

    if ' OR ' in rule:
        parts = split_rule(rule, 'OR')
        left = create_rule(parts[0].strip())
        right = create_rule(parts[1].strip())
        return Node('operator', 'OR', left, right)

    # Check for comparison operators
    if '>' in rule:
        variable, value = rule.split('>', 1)  # Split on first '>'
        return Node('comparison', '>', Node('operand', variable.strip()), Node('operand', value.strip()))
    elif '<' in rule:
        variable, value = rule.split('<', 1)  # Split on first '<'
        return Node('comparison', '<', Node('operand', variable.strip()), Node('operand', value.strip()))
    elif '=' in rule:
        variable, value = rule.split('=', 1)  # Split on first '='
        return Node('comparison', '==', Node('operand', variable.strip()), Node('operand', value.strip()))

    # If it's just an operand
    return Node('operand', rule.strip())

    def parse_expression(index):
        """Parse the expression recursively."""
        if index >= len(tokens):
            return None, index
        
        token = tokens[index]
        if token == '(':
            left_node, index = parse_expression(index + 1)
            operator = tokens[index]
            index += 1  # Skip operator
            right_node, index = parse_expression(index)
            index += 1  # Skip ')'
            return Node("operator", operator, left_node, right_node), index
        elif token.isidentifier() or re.match(r'\w+', token):  # Check for variable names
            return Node("operand", token), index + 1
        else:
            return None, index

    ast, _ = parse_expression(0)
    return ast

def combine_rules(rules):
    """Combine multiple rules into a single AST."""
    combined = None
    for rule in rules:
        if combined is None:
            combined = create_rule(rule)
        else:
            combined = Node("operator", "OR", combined, create_rule(rule))
    return combined


In [11]:
def evaluate_rule(ast, data):
    if ast.type == "operand":
        return data.get(ast.value, 0)  # Return the value associated with the operand

    if ast.type == "comparison":
        left_value = evaluate_rule(ast.left, data)
        right_value = evaluate_rule(ast.right, data)
        if ast.value == '>':
            return left_value > right_value
        elif ast.value == '<':
            return left_value < right_value
        elif ast.value == '==':
            return left_value == right_value
        elif ast.value == '!=':
            return left_value != right_value
        elif ast.value == '>=':
            return left_value >= right_value
        elif ast.value == '<=':
            return left_value <= right_value

    if ast.type == "operator":
        left_result = evaluate_rule(ast.left, data)
        right_result = evaluate_rule(ast.right, data)
        if ast.value == "AND":
            return left_result and right_result
        elif ast.value == "OR":
            return left_result or right_result


In [13]:
# Sample JSON data for evaluation
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}

# Create individual rules and verify AST
rule1 = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
ast1 = create_rule(rule1)
print("AST for Rule 1:")
print(ast1)

# Combine rules
rule2 = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"
combined_ast = combine_rules([rule1, rule2])
print("Combined AST:")
print(combined_ast)

# Evaluate combined rule
result = evaluate_rule(combined_ast, data)



Creating rule: ((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)
Creating rule: (age > 30
Creating rule: department = 'Sales') OR (age < 25
Creating rule: department = 'Sales')
Creating rule: (age < 25
AST for Rule 1:
Node(type=operator, value=AND, left=Node(type=comparison, value=>, left=Node(type=operand, value=(age, left=None, right=None), right=Node(type=operand, value=30, left=None, right=None)), right=Node(type=operator, value=OR, left=Node(type=comparison, value===, left=Node(type=operand, value=department, left=None, right=None), right=Node(type=operand, value='Sales'), left=None, right=None)), right=Node(type=comparison, value=<, left=Node(type=operand, value=(age, left=None, right=None), right=Node(type=operand, value=25, left=None, right=None))))
Creating rule: ((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)
Creating rule: (age > 30
Cr

                         Application 2 : Real-Time Data Processing System for
                              Weather Monitoring with Rollups and Aggregates

In [ ]:
import requests
import time
import json

# Your OpenWeatherMap API key
API_KEY = '6a0666fb9b431506dbe62cff67fdfcd0'  # api
CITIES = ['Delhi', 'Mumbai', 'Chennai', 'Bangalore', 'Kolkata', 'Hyderabad']

def get_weather_data(city):
    """Fetch weather data from OpenWeatherMap API."""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching weather data for {city}: {response.status_code} - {response.text}")
        return {}  # Return an empty dictionary to avoid further errors

def process_weather_data(data):
    """Process the raw weather data and return essential information."""
    # Check if the necessary keys exist before accessing them
    if 'main' in data and 'weather' in data and 'dt' in data:
        main = data['main']
        weather_condition = data['weather'][0]['main']
        temp = main['temp']
        feels_like = main['feels_like']
        return {
            'main': weather_condition,
            'temp': temp,
            'feels_like': feels_like,
            'timestamp': data['dt']
        }
    else:
        print("Incomplete weather data received. Skipping processing. Check if API Key and City are correct.")
        return {}  # Return an empty dictionary to avoid further errors

def main():
    """Main loop for fetching and processing weather data."""
    daily_summary = {}

    while True:
        for city in CITIES:
            weather_data = get_weather_data(city)
            processed_data = process_weather_data(weather_data)

            if processed_data:  # Only update if data was processed successfully
                day = time.strftime('%Y-%m-%d', time.localtime(processed_data['timestamp']))
                if day not in daily_summary:
                    daily_summary[day] = {
                        'temperatures': [],
                        'conditions': []
                    }

                # Append data to daily summary
                daily_summary[day]['temperatures'].append(processed_data['temp'])
                daily_summary[day]['conditions'].append(processed_data['main'])

        # Print daily aggregates at the end of each day
        if time.strftime('%H:%M') == '23:59':  # Check at the end of the day
            for day in daily_summary:
                temps = daily_summary[day]['temperatures']
                conditions = daily_summary[day]['conditions']
                average_temp = sum(temps) / len(temps)
                max_temp = max(temps)
                min_temp = min(temps)
                dominant_condition = max(set(conditions), key=conditions.count)  # Most common condition

                print(f"Daily aggregates for {day}:")
                print(f"  Average Temperature: {average_temp:.2f}°C")
                print(f"  Max Temperature: {max_temp:.2f}°C")
                print(f"  Min Temperature: {min_temp:.2f}°C")
                print(f"  Dominant Condition: {dominant_condition}")

        time.sleep(300)  # Sleep for 5 minutes

if __name__ == "__main__":
    main()

# Rule Engine and Weather Monitoring System

## Overview
This project consists of two applications:
1. A Rule Engine using Abstract Syntax Tree (AST) for user eligibility evaluation based on various attributes.
2. A Real-Time Weather Monitoring System using data from the OpenWeatherMap API.

## Dependencies
- Python 3.x
- Requests library: Install with `pip install requests`

## Application 1: Rule Engine with AST
### Features
- Create rules using a string representation.
- Combine multiple rules into a single AST.
- Evaluate rules against user attributes.

### Usage
1. Define your rules using the specified syntax.
2. Use the `create_rule` function to generate AST nodes.
3. Combine rules with `combine_rules`.
4. Evaluate rules against user data using `evaluate_rule`.

## Application 2: Real-Time Weather Monitoring System
### Features
- Retrieve weather data from the OpenWeatherMap API.
- Process and analyze weather data continuously.
- Generate daily summaries and alerts.

### Usage
1. Replace `your_api_key` with your OpenWeatherMap API key.
2. Run the script to start monitoring weather conditions.
3. Daily summaries will be printed at the end of each day.

## Running the Applications
1. Clone the repository.
2. Install dependencies.
3. Run the Python script to start the applications.

